In [1]:
!pip install git+https://github.com/huggingface/trl.git

  Cloning https://github.com/huggingface/trl.git to /tmp/pip-req-build-m8oja4jw
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl.git /tmp/pip-req-build-m8oja4jw
  Resolved https://github.com/huggingface/trl.git to commit e622196097109080b73584d598d4162e64fc6bea
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
!pip install -q -U trl accelerate bitsandbytes

In [3]:
# This notebook has been adapted form the PPO example found here:
# https://github.com/huggingface/trl/blob/1b1242cc6522feb4eb063feb20097a79b11b127a/examples/scripts/ppo/ppo_tldr.py

# Copyright 2020-2025 The HuggingFace Team. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# /// script
# dependencies = [
#     "trl",
#     "peft",
#     "trackio",
#     "kernels",
# ]
# ///

import os
import shutil

import torch
from accelerate import PartialState
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    HfArgumentParser,
)

from trl import ModelConfig, ScriptArguments, get_kbit_device_map, get_peft_config, get_quantization_config
from trl.experimental.ppo import PPOConfig, PPOTrainer

/tmp/ipython-input-1498927528.py:41: UserWarning: You are importing from 'trl.experimental'. APIs here are unstable and may change or be removed without notice. Silence this warning by setting environment variable TRL_EXPERIMENTAL_SILENCE=1.
  from trl.experimental.ppo import PPOConfig, PPOTrainer


In [4]:
from dataclasses import fields
print([f.name for f in fields(ScriptArguments)])
print("dataset_name" in [f.name for f in fields(ScriptArguments)])

print([f.name for f in fields(PPOConfig)])
print("eval_steps" in [f.name for f in fields(PPOConfig)])

print([f.name for f in fields(ModelConfig)])
print("load_in_4bit" in [f.name for f in fields(ModelConfig)])

['dataset_name', 'dataset_config', 'dataset_train_split', 'dataset_test_split', 'dataset_streaming', 'gradient_checkpointing_use_reentrant', 'ignore_bias_buffers']
True
['output_dir', 'overwrite_output_dir', 'do_train', 'do_eval', 'do_predict', 'eval_strategy', 'prediction_loss_only', 'per_device_train_batch_size', 'per_device_eval_batch_size', 'per_gpu_train_batch_size', 'per_gpu_eval_batch_size', 'gradient_accumulation_steps', 'eval_accumulation_steps', 'eval_delay', 'torch_empty_cache_steps', 'learning_rate', 'weight_decay', 'adam_beta1', 'adam_beta2', 'adam_epsilon', 'max_grad_norm', 'num_train_epochs', 'max_steps', 'lr_scheduler_type', 'lr_scheduler_kwargs', 'warmup_ratio', 'warmup_steps', 'log_level', 'log_level_replica', 'log_on_each_node', 'logging_dir', 'logging_strategy', 'logging_first_step', 'logging_steps', 'logging_nan_inf_filter', 'save_strategy', 'save_steps', 'save_total_limit', 'save_safetensors', 'save_on_each_node', 'save_only_model', 'restore_callback_states_from_c

In [5]:
# Enable logging in a Hugging Face Space
os.environ.setdefault("TRACKIO_SPACE_ID", "trl-trackio")

# Instead of script arguments. I will set global variables

# ModelConfig
model_name_or_path = "openai-community/gpt2" #"EleutherAI/pythia-1b-deduped"

# PPOConfig
reward_model_path = "asudeekiz/gpt2-imdb-positive-20k-with_reward-10it" #"cleanrl/EleutherAI_pythia-1b-deduped__reward__tldr"
output_dir = "/content/gpt2-trl-style-ppo"
eval_strategy = "steps"
learning_rate = 3e-6
per_device_train_batch_size = 1
gradient_accumulation_steps = 4 # 64
total_episodes = 2 #30000
sft_model_path = "Holarissun/gpt2-sft-tldr"
missing_eos_penalty = 1.0
response_length = 33 #53
eval_steps = 100
stop_token = "eos"

# ScriptArguments
dataset_name = "trl-internal-testing/tldr-preference-sft-trl-style"
dataset_test_split = "validation"

script_args = ScriptArguments(
    dataset_name=dataset_name,
    dataset_test_split=dataset_test_split
)

training_args = PPOConfig(
    learning_rate=learning_rate,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    total_episodes=total_episodes,
    eval_strategy=eval_strategy,
    eval_steps=eval_steps,
    sft_model_path=sft_model_path,
    reward_model_path=reward_model_path,
    output_dir=output_dir,
    stop_token=stop_token,
    response_length=response_length,
    missing_eos_penalty=missing_eos_penalty,
    bf16=True
)

model_args = ModelConfig(
    model_name_or_path=model_name_or_path,
    load_in_4bit=True
)

#parser = HfArgumentParser((ScriptArguments, PPOConfig, ModelConfig))
#script_args, training_args, model_args = parser.parse_args_into_dataclasses()


"""
python examples/scripts/ppo/ppo_tldr.py \
    --dataset_name trl-internal-testing/tldr-preference-sft-trl-style \
    --dataset_test_split validation \
    --learning_rate 3e-6 \
    --output_dir pythia-1b-deduped-tldr-preference-sft-trl-style-ppo \
    --per_device_train_batch_size 1 \
    --gradient_accumulation_steps 64 \
    --total_episodes 30000 \
    --model_name_or_path EleutherAI/pythia-1b-deduped \
    --sft_model_path cleanrl/EleutherAI_pythia-1b-deduped__sft__tldr \
    --reward_model_path cleanrl/EleutherAI_pythia-1b-deduped__reward__tldr \
    --missing_eos_penalty 1.0 \
    --stop_token eos \
    --response_length 53 \
    --eval_strategy steps \
    --eval_steps 100

accelerate launch --config_file examples/accelerate_configs/deepspeed_zero2.yaml \
    examples/scripts/ppo/ppo_tldr.py \
    --dataset_name trl-internal-testing/tldr-preference-sft-trl-style \
    --dataset_test_split validation \
    --output_dir pythia-1b-deduped-tldr-preference-sft-trl-style-ppo \
    --learning_rate 3e-6 \
    --per_device_train_batch_size 16 \
    --gradient_accumulation_steps 4 \
    --total_episodes 1000000 \
    --model_name_or_path EleutherAI/pythia-1b-deduped \
    --sft_model_path cleanrl/EleutherAI_pythia-1b-deduped__sft__tldr \
    --reward_model_path cleanrl/EleutherAI_pythia-1b-deduped__reward__tldr \
    --local_rollout_forward_batch_size 16 \
    --missing_eos_penalty 1.0 \
    --stop_token eos \
    --eval_strategy steps \
    --eval_steps 100
"""

'\npython examples/scripts/ppo/ppo_tldr.py     --dataset_name trl-internal-testing/tldr-preference-sft-trl-style     --dataset_test_split validation     --learning_rate 3e-6     --output_dir pythia-1b-deduped-tldr-preference-sft-trl-style-ppo     --per_device_train_batch_size 1     --gradient_accumulation_steps 64     --total_episodes 30000     --model_name_or_path EleutherAI/pythia-1b-deduped     --sft_model_path cleanrl/EleutherAI_pythia-1b-deduped__sft__tldr     --reward_model_path cleanrl/EleutherAI_pythia-1b-deduped__reward__tldr     --missing_eos_penalty 1.0     --stop_token eos     --response_length 53     --eval_strategy steps     --eval_steps 100\n\naccelerate launch --config_file examples/accelerate_configs/deepspeed_zero2.yaml     examples/scripts/ppo/ppo_tldr.py     --dataset_name trl-internal-testing/tldr-preference-sft-trl-style     --dataset_test_split validation     --output_dir pythia-1b-deduped-tldr-preference-sft-trl-style-ppo     --learning_rate 3e-6     --per_devic

In [6]:
def prepare_dataset(dataset, tokenizer):
        """pre-tokenize the dataset before training; only collate during training"""

        def tokenize(element):
            # tokenizer.chat_template = "{% for message in messages %}{% if message['role'] == 'user' %}{{ '[INST] ' + message['content'] + ' [/INST]' }}{% elif message['role'] == 'assistant' %}{{ message['content'] }}{% endif %}{% endfor %}"
            input_ids = tokenizer.apply_chat_template(
                element["messages"][:1],
                padding=False,
                add_generation_prompt=True,
            )
            return {"input_ids": input_ids, "lengths": len(input_ids)}

        return dataset.map(
            tokenize,
            remove_columns=dataset.column_names,
            num_proc=training_args.dataset_num_proc,
        )

In [21]:
# remove output_dir if exists
shutil.rmtree(training_args.output_dir, ignore_errors=True)

################
# Model & Tokenizer
################
dtype = model_args.dtype if model_args.dtype in ["auto", None] else getattr(torch, model_args.dtype)
model_kwargs = dict(
    revision=model_args.model_revision,
    attn_implementation=model_args.attn_implementation,
    dtype=dtype,
)
quantization_config = get_quantization_config(model_args)
if quantization_config is not None:
    # Passing None would not be treated the same as omitting the argument, so we include it only when valid.
    model_kwargs["device_map"] = get_kbit_device_map()
    model_kwargs["quantization_config"] = quantization_config

tokenizer = AutoTokenizer.from_pretrained(
    model_args.model_name_or_path, padding_side="left", trust_remote_code=model_args.trust_remote_code
)
tokenizer.add_special_tokens({"pad_token": "[PAD]"})
# Explicitly set pad_token_id to suppress the warning during generation
tokenizer.pad_token_id = tokenizer.pad_token_id or tokenizer.eos_token_id # Set a chat template for the tokenizer to enable apply_chat_template
tokenizer.chat_template = tokenizer.chat_template = "{% for message in messages %}{% if message['role'] == 'user' %}{{ '<INST> ' + message['content'] + ' </INST>' }}{% elif message['role'] == 'assistant' %}{{ message['content'] }}{% endif %}{% endfor %}"
#tokenizer.chat_template = tokenizer.chat_template = "{% for message in messages %}{% if message['role'] == 'user' %}{{ '[INST] ' + message['content'] + ' [/INST]' }}{% elif message['role'] == 'assistant' %}{{ message['content'] }}{% endif %}{% endfor %}"
value_model = AutoModelForSequenceClassification.from_pretrained(
    training_args.reward_model_path, trust_remote_code=model_args.trust_remote_code, num_labels=1
)
# value_model.config.use_cache = False # Add this line to prevent the TypeError
reward_model = AutoModelForSequenceClassification.from_pretrained(
    training_args.reward_model_path, trust_remote_code=model_args.trust_remote_code, num_labels=1
)
# reward_model.config.use_cache = False # Add this line to prevent the TypeError
policy = AutoModelForCausalLM.from_pretrained(
    training_args.sft_model_path, trust_remote_code=model_args.trust_remote_code
)

peft_config = get_peft_config(model_args)
if peft_config is None:
    ref_policy = AutoModelForCausalLM.from_pretrained(
        training_args.sft_model_path, trust_remote_code=model_args.trust_remote_code
    )
else:
    ref_policy = None

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at asudeekiz/gpt2-imdb-positive-20k-with_reward-10it and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at asudeekiz/gpt2-imdb-positive-20k-with_reward-10it and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
dataset = load_dataset(script_args.dataset_name, name=script_args.dataset_config)
train_dataset = dataset[script_args.dataset_train_split]
eval_dataset = dataset[script_args.dataset_test_split] if training_args.eval_strategy != "no" else None

# Compute that only on the main process for faster data processing.
# see: https://github.com/huggingface/trl/pull/1255
with PartialState().local_main_process_first():
    train_dataset = prepare_dataset(train_dataset, tokenizer)
    if eval_dataset is not None:
        eval_dataset = prepare_dataset(eval_dataset, tokenizer)
    # filtering
    train_dataset = train_dataset.filter(lambda x: x["lengths"] <= 256, num_proc=training_args.dataset_num_proc)
    if eval_dataset is not None:
        eval_dataset = eval_dataset.filter(lambda x: x["lengths"] <= 256, num_proc=training_args.dataset_num_proc)

assert train_dataset[0]["input_ids"][-1] != tokenizer.eos_token_id, "The last token should not be an EOS token"


Map:   0%|          | 0/116722 [00:00<?, ? examples/s]

Map:   0%|          | 0/6447 [00:00<?, ? examples/s]

Filter:   0%|          | 0/116722 [00:00<?, ? examples/s]

Filter:   0%|          | 0/6447 [00:00<?, ? examples/s]

In [23]:
################
# Training
################
trainer = PPOTrainer(
    args=training_args,
    processing_class=tokenizer,
    model=policy,
    ref_model=ref_policy,
    reward_model=reward_model,
    value_model=value_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
)
trainer.train()

# Save and push to hub
trainer.save_model(training_args.output_dir)
if training_args.push_to_hub:
    trainer.push_to_hub(dataset_name=script_args.dataset_name)

trainer.generate_completions()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


===training policy===


Step,Training Loss,Validation Loss


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ query                                       ┃ model response                               ┃ score              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ <INST> SUBREDDIT: r/AskReddit               │  SUBREDDIT: r/AskReddit                      │ 3.6724982261657715 │
│                                             │                                              │                    │
│ TITLE: How do you get someone out of your   │ EDIT: I'm sorry for the long post, I just    │                    │
│ head?                                       │ want to make it clear that I'm not a good    │                    │
│                                             │                                              │                    │
│ POST: Hi,                                   │                                              │                    │
│ I'm 22, and I have been with my girlfriend  │                                              │                    │
│ for 5 years now. We recently moved          │                                              │                    │
│ together. We've always loved each other     │                                              │                    │
│ intensely.                                  │                                              │                    │
│                                             │                                              │                    │
│ Problem, I recently started to have         │                                              │                    │
│ feelings for an other person (a friend).    │                                              │                    │
│ This person has had a boyfriend for now 3   │                                              │                    │
│ years, and has absolutely no ideas. Those   │                                              │                    │
│ feelings were so strong, it was hard to     │                                              │                    │
│ hide them. After 2 months of me being       │                                              │                    │
│ distant and really sad, my girlfriend       │                                              │                    │
│ forced me to say what was bothering me. I'm │                                              │                    │
│ not a good liar, and now she knows.         │                                              │                    │
│                                             │                                              │                    │
│ We decided to give us a week alone, I went  │                                              │                    │
│ to my parents.                              │                                              │                    │
│                                             │                                              │                    │
│ Now, I'm completely lost. I keep on         │                                              │                    │
│ thinking about this person, and I hate      │                                              │                    │
│ that. I would like for those feelings to go │                                              │                    │
│ away, to leave me alone. But I can't.       │                                              │                    │
│                                             │                                              │                    │
│ What do I do? It's been 3 months now, and   │                                              │                    │
│ I'm just desperate.                         │                                              │                    │
│                                             │         

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ query                                       ┃ model response                               ┃ score              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ <INST> SUBREDDIT: r/AskReddit               │  SUBREDDIT: r/AskReddit                      │ 3.6724982261657715 │
│                                             │                                              │                    │
│ TITLE: How do you get someone out of your   │ EDIT: I'm sorry for the long post, I just    │                    │
│ head?                                       │ want to make it clear that I'm not a good    │                    │
│                                             │                                              │                    │
│ POST: Hi,                                   │                                              │                    │
│ I'm 22, and I have been with my girlfriend  │                                              │                    │
│ for 5 years now. We recently moved          │                                              │                    │
│ together. We've always loved each other     │                                              │                    │
│ intensely.                                  │                                              │                    │
│                                             │                                              │                    │
│ Problem, I recently started to have         │                                              │                    │
│ feelings for an other person (a friend).    │                                              │                    │
│ This person has had a boyfriend for now 3   │                                              │                    │
│ years, and has absolutely no ideas. Those   │                                              │                    │
│ feelings were so strong, it was hard to     │                                              │                    │
│ hide them. After 2 months of me being       │                                              │                    │
│ distant and really sad, my girlfriend       │                                              │                    │
│ forced me to say what was bothering me. I'm │                                              │                    │
│ not a good liar, and now she knows.         │                                              │                    │
│                                             │                                              │                    │
│ We decided to give us a week alone, I went  │                                              │                    │
│ to my parents.                              │                                              │                    │
│                                             │                                              │                    │
│ Now, I'm completely lost. I keep on         │                                              │                    │
│ thinking about this person, and I hate      │                                              │                    │
│ that. I would like for those feelings to go │                                              │                    │
│ away, to leave me alone. But I can't.       │                                              │                    │
│                                             │                                              │                    │
│ What do I do? It's been 3 months now, and   │                                              │                    │
│ I'm just desperate.                         │                                              │                    │
│                                             │         